In [1]:
import numpy as np
import pandas as pd
# library for splitting training-testing
from sklearn.model_selection import train_test_split
# library for classification
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler
from sklearn import pipeline
from lightgbm import LGBMClassifier

from hyperopt import tpe,hp,Trials
from hyperopt.fmin import fmin
import warnings

In [2]:
x_train = pd.read_feather("Data/x_train__IterativeImputer_genFeats.ftr")
x_test = pd.read_feather("Data/x_test__IterativeImputer_genFeats.ftr")

y_train = pd.read_csv("Data/train_y.csv")
y_test = pd.read_csv("Data/Sample_Output.csv", names = ['ID', 'Target'], header=None)

In [3]:
x_train_1, x_val, y_train_1, y_val = train_test_split(x_train, y_train, test_size=0.15, random_state=59, stratify=y_train)

In [ ]:
#check if given parameter can be interpreted as a numerical value
def is_number(s):
    if s is None:
        return False
    try:
        float(s)
        return True
    except ValueError:
        return False

def convert_int_params(names, params):
    for int_type in names:
        #sometimes the parameters can be choices between options or numerical values. like "log2" vs "1-10"
        raw_val = params[int_type]
        if is_number(raw_val):
            params[int_type] = int(raw_val)
    return params

def create_fit_params(params):
    using_dart = params['boosting_type'] == "dart"
    
    fit_params = {"eval_metric": "multi_logloss"}
    if using_dart:
        n_estimators = 75
    else:
        n_estimators = 250
        fit_params["early_stopping_rounds"] = 10
    params["n_estimators"] = n_estimators
    return fit_params

def objective(params):
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    
    int_types = ["num_leaves", "min_child_samples", "subsample_for_bin", "min_data_in_leaf"]
    params = convert_int_params(int_types, params)

    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    #    print("running with params:"+str(params))

    fit_params = create_fit_params(params)
    
    model = LGBMClassifier(**params)
    
    model.fit(x_train_1, 
              y_train_1.values.reshape(-1), 
              eval_set=[(x_train_1, y_train_1.values.reshape(-1)), (x_val, y_val.values.reshape(-1))], 
              verbose=1, **fit_params)
    
    preds_tr = model.predict(x_train_1,)
    preds_val = model.predict(x_val)
    val_score = accuracy_score(y_val, preds_val)
    tr_score = accuracy_score(y_train_1, preds_tr)
    print(tr_score, val_score, params)
    return -1.0*val_score

def optimize(trial):
    space = {
        #this is just piling on most of the possible parameter values for LGBM
        #some of them apparently don't make sense together, but works for now.. :)
        'boosting_type': hp.choice('boosting_type',
                                   [{'boosting_type': 'gbdt',
#                                     'subsample': hp.uniform('dart_subsample', 0.5, 1)
                                     },
                                    {'boosting_type': 'dart',
#                                     'subsample': hp.uniform('dart_subsample', 0.5, 1)
                                     },
                                    {'boosting_type': 'goss'}]),
        'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
        'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
        'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1), #alias "subsample"
        'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 4, 15, 1),
        'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
        'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)]),
        'verbose': -1,
        #the LGBM parameters docs list various aliases, and the LGBM implementation seems to complain about
        #the following not being used due to other params, so trying to silence the complaints by setting to None
        'subsample': None, #overridden by bagging_fraction
        'reg_alpha': None, #overridden by lambda_l1
        'reg_lambda': None, #overridden by lambda_l2
        'min_sum_hessian_in_leaf': None, #overrides min_child_weight
        'min_child_samples': None, #overridden by min_data_in_leaf
        'colsample_bytree': None, #overridden by feature_fraction
#        'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
        'min_child_weight': hp.loguniform('min_child_weight', -16, 5), #also aliases to min_sum_hessian
#        'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
#        'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
#        'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
        'objective' : "multiclass",
        "num_class" : 4,
        'num_threads' : 8
    }
    best=fmin(fn=objective,space=space,algo=tpe.suggest,trials=trial,max_evals=100,rstate=np.random.default_rng(59))
    return best

trial=Trials()
best=optimize(trial)

  0%|                                                                  | 0/100 [00:00<?, ?trial/s, best loss=?]

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[1]	valid_0's multi_logloss: 0.828324	valid_1's multi_logloss: 0.828513                                        
[2]	valid_0's multi_logloss: 0.804925	valid_1's multi_logloss: 0.805307                                        
[3]	valid_0's multi_logloss: 0.78456	valid_1's multi_logloss: 0.785053                                         
[4]	valid_0's multi_logloss: 0.766368	valid_1's multi_logloss: 0.767002                                        
[5]	valid_0's multi_logloss: 0.749516	valid_1's multi_logloss: 0.750314                                        
[6]	valid_0's multi_logloss: 0.734139	valid_1's multi_logloss: 0.735056                                        
[7]	valid_0's multi_logloss: 0.71972	valid_1's multi_logloss: 0.720838                                         
[8]	valid_0's multi_logloss: 0.706617	valid_1's multi_logloss: 0.707896                                        
[9]	valid_0's multi_logloss: 0.69432	valid_1's multi_logloss: 0.695763                                  

[147]	valid_0's multi_logloss: 0.432642	valid_1's multi_logloss: 0.456562                                      
[148]	valid_0's multi_logloss: 0.432356	valid_1's multi_logloss: 0.456467                                      
[149]	valid_0's multi_logloss: 0.432076	valid_1's multi_logloss: 0.456361                                      
[150]	valid_0's multi_logloss: 0.43181	valid_1's multi_logloss: 0.456254                                       
[151]	valid_0's multi_logloss: 0.431545	valid_1's multi_logloss: 0.4562                                        
[152]	valid_0's multi_logloss: 0.431276	valid_1's multi_logloss: 0.456121                                      
[153]	valid_0's multi_logloss: 0.431012	valid_1's multi_logloss: 0.456048                                      
[154]	valid_0's multi_logloss: 0.430738	valid_1's multi_logloss: 0.455954                                      
[155]	valid_0's multi_logloss: 0.430479	valid_1's multi_logloss: 0.45587                                

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



...

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8             
[1]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                           
[2]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                           
[3]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                           
[4]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                           
[5]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                           
[6]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                           
[7]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                           
[8]	valid_0's multi_logloss: 1.14992	valid_1's multi_logloss: 1.1499                                    

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8             
[1]	valid_0's multi_logloss: 0.795149	valid_1's multi_logloss: 0.795352                                        
[2]	valid_0's multi_logloss: 0.752585	valid_1's multi_logloss: 0.752981                                        
[3]	valid_0's multi_logloss: 0.718076	valid_1's multi_logloss: 0.718599                                        
[4]	valid_0's multi_logloss: 0.689406	valid_1's multi_logloss: 0.690085                                        
[5]	valid_0's multi_logloss: 0.665447	valid_1's multi_logloss: 0.666245                                        
[6]	valid_0's multi_logloss: 0.644132	valid_1's multi_logloss: 0.645143                                        
[7]	valid_0's multi_logloss: 0.625605	valid_1's multi_logloss: 0.626779                                        
[8]	valid_0's multi_logloss: 0.609362	valid_1's multi_logloss: 0.610793                                 

[146]	valid_0's multi_logloss: 0.412284	valid_1's multi_logloss: 0.44868                                       
[147]	valid_0's multi_logloss: 0.411988	valid_1's multi_logloss: 0.448684                                      
[148]	valid_0's multi_logloss: 0.411682	valid_1's multi_logloss: 0.44864                                       
[149]	valid_0's multi_logloss: 0.411385	valid_1's multi_logloss: 0.448622                                      
[150]	valid_0's multi_logloss: 0.411086	valid_1's multi_logloss: 0.448574                                      
[151]	valid_0's multi_logloss: 0.410802	valid_1's multi_logloss: 0.448532                                      
[152]	valid_0's multi_logloss: 0.410505	valid_1's multi_logloss: 0.448492                                      
[153]	valid_0's multi_logloss: 0.410215	valid_1's multi_logloss: 0.44844                                       
[154]	valid_0's multi_logloss: 0.409933	valid_1's multi_logloss: 0.448426                               

0.8408130723243675                                                                                               
0.818920057527202                                                                                                
{'bagging_fraction': 0.7735281708228888, 'boosting_type': 'gbdt', 'colsample_bytree': None, 'feature_fraction': 0.7019688791355287, 'lambda_l1': 0.0005066902069864062, 'lambda_l2': 3.374501183311695e-06, 'learning_rate': 0.04616756254060491, 'min_child_samples': None, 'min_child_weight': 3.29982863252954, 'min_data_in_leaf': 56, 'min_sum_hessian_in_leaf': None, 'num_class': 4, 'num_leaves': 58, 'num_threads': 8, 'objective': 'multiclass', 'reg_alpha': None, 'reg_lambda': None, 'subsample': None, 'subsample_for_bin': 140000, 'verbose': -1, 'n_estimators': 200}

In [4]:
bestParams = {'bagging_fraction': 0.7735281708228888, 'boosting_type': 'gbdt', 'colsample_bytree': None, 'feature_fraction': 0.7019688791355287, 'lambda_l1': 0.0005066902069864062, 'lambda_l2': 3.374501183311695e-06, 'learning_rate': 0.04616756254060491, 'min_child_samples': None, 'min_child_weight': 3.29982863252954, 'min_data_in_leaf': 56, 'min_sum_hessian_in_leaf': None, 'num_class': 4, 'num_leaves': 58, 'num_threads': 8, 'objective': 'multiclass', 'reg_alpha': None, 'reg_lambda': None, 'subsample': None, 'subsample_for_bin': 140000, 'verbose': -1, 'n_estimators': 200}

In [5]:
model = LGBMClassifier(**bestParams)
    
model.fit(x_train_1, 
          y_train_1.values.reshape(-1), 
          eval_set=[(x_train_1, y_train_1.values.reshape(-1)), (x_val, y_val.values.reshape(-1))], 
          verbose=1, early_stopping_rounds=10)

preds_tr = model.predict(x_train_1,)
preds_val = model.predict(x_val)
val_score = accuracy_score(y_val, preds_val)
tr_score = accuracy_score(y_train_1, preds_tr)
print(tr_score, val_score)

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8
[1]	valid_0's multi_logloss: 0.810241	valid_1's multi_logloss: 0.810551
[2]	valid_0's multi_logloss: 0.774345	valid_1's multi_logloss: 0.774867
[3]	valid_0's multi_logloss: 0.744897	valid_1's multi_logloss: 0.745565
[4]	valid_0's multi_logloss: 0.719764	valid_1's multi_logloss: 0.720506
[5]	valid_0's multi_logloss: 0.697359	valid_1's multi_logloss: 0.698334
[6]	valid_0's multi_logloss: 0.677902	valid_1's multi_logloss: 0.679041
[7]	valid_0's multi_logloss: 0.660172	valid_1's multi_logloss: 0.661511
[8]	valid_0's multi_logloss: 0.644577	valid_1's multi_logloss: 0.646086
[9]	valid_0's multi_logloss: 0.630197	valid_1's multi_logloss: 0.631898
[10]	valid_0's multi_logloss: 0.617135	valid_1's multi_logloss: 0.618995
[11]	valid_0's multi_logloss: 0.605308	valid_1's multi_logloss: 0.607348
[12]	valid_0's multi_logloss: 0.594541	valid_1's multi_logloss: 0.596755
[13]	valid_0's multi_logloss: 0.58

[113]	valid_0's multi_logloss: 0.426066	valid_1's multi_logloss: 0.452555
[114]	valid_0's multi_logloss: 0.425721	valid_1's multi_logloss: 0.452449
[115]	valid_0's multi_logloss: 0.425373	valid_1's multi_logloss: 0.452378
[116]	valid_0's multi_logloss: 0.424987	valid_1's multi_logloss: 0.452266
[117]	valid_0's multi_logloss: 0.42465	valid_1's multi_logloss: 0.452164
[118]	valid_0's multi_logloss: 0.424297	valid_1's multi_logloss: 0.452094
[119]	valid_0's multi_logloss: 0.423957	valid_1's multi_logloss: 0.45201
[120]	valid_0's multi_logloss: 0.423597	valid_1's multi_logloss: 0.451915
[121]	valid_0's multi_logloss: 0.423242	valid_1's multi_logloss: 0.451854
[122]	valid_0's multi_logloss: 0.422899	valid_1's multi_logloss: 0.451811
[123]	valid_0's multi_logloss: 0.422558	valid_1's multi_logloss: 0.45171
[124]	valid_0's multi_logloss: 0.422217	valid_1's multi_logloss: 0.451643
[125]	valid_0's multi_logloss: 0.421875	valid_1's multi_logloss: 0.451591
[126]	valid_0's multi_logloss: 0.421539	v

In [10]:
probs_tr = model.predict_proba(x_train_1)
probs_val = model.predict_proba(x_val)

In [6]:
# np.save("Preds/LGB_val", probs_val)
# np.save("Preds/LGB_tr", probs_tr)

probs_test = model.predict_proba(x_test)
np.save("Preds/LGB_test", probs_test)

In [6]:
#check if given parameter can be interpreted as a numerical value
# def is_number(s):
#     if s is None:
#         return False
#     try:
#         float(s)
#         return True
#     except ValueError:
#         return False

# def convert_int_params(names, params):
#     for int_type in names:
#         #sometimes the parameters can be choices between options or numerical values. like "log2" vs "1-10"
#         raw_val = params[int_type]
#         if is_number(raw_val):
#             params[int_type] = int(raw_val)
#     return params

# def create_fit_params(params):
#     using_dart = params['boosting_type'] == "dart"
    
#     fit_params = {"eval_metric": "multi_logloss"}
#     if using_dart:
#         n_estimators = 75
#     else:
#         n_estimators = 200
#         fit_params["early_stopping_rounds"] = 10
#     params["n_estimators"] = n_estimators
#     return fit_params

def objective(params):
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)
    
    model = LGBMClassifier(**bestParams, random_state=int(params['random_state']))
    
    model.fit(x_train_1, 
              y_train_1.values.reshape(-1), 
              eval_set=[(x_train_1, y_train_1.values.reshape(-1)), (x_val, y_val.values.reshape(-1))], 
              verbose=1, early_stopping_rounds=10)
    
    preds_tr = model.predict(x_train_1,)
    preds_val = model.predict(x_val)
    val_score = accuracy_score(y_val, preds_val)
    tr_score = accuracy_score(y_train_1, preds_tr)
    print(tr_score, val_score, params)
    return -1.0*val_score

def optimize(trial):
    space = {
        #this is just piling on most of the possible parameter values for LGBM
        #some of them apparently don't make sense together, but works for now.. :)
        'random_state': hp.quniform('random_state', 1, 100, 1),
    }
    best=fmin(fn=objective,space=space,algo=tpe.suggest,trials=trial,max_evals=20,rstate=np.random.default_rng(59))
    return best

trial=Trials()
best=optimize(trial)

  0%|                                                                   | 0/20 [00:00<?, ?trial/s, best loss=?]

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8             
[1]	valid_0's multi_logloss: 0.809959	valid_1's multi_logloss: 0.810152                                        
[2]	valid_0's multi_logloss: 0.774792	valid_1's multi_logloss: 0.7752                                          
[3]	valid_0's multi_logloss: 0.744832	valid_1's multi_logloss: 0.745513                                        
[4]	valid_0's multi_logloss: 0.719346	valid_1's multi_logloss: 0.720251                                        
[5]	valid_0's multi_logloss: 0.697259	valid_1's multi_logloss: 0.698297                                        
[6]	valid_0's multi_logloss: 0.677741	valid_1's multi_logloss: 0.678944                                        
[7]	valid_0's multi_logloss: 0.660198	valid_1's multi_logloss: 0.661635                                        
[8]	valid_0's multi_logloss: 0.644476	valid_1's multi_logloss: 0.646099                                 

[73]	valid_0's multi_logloss: 0.44351	valid_1's multi_logloss: 0.459392                                        
[74]	valid_0's multi_logloss: 0.442967	valid_1's multi_logloss: 0.459083                                       
[75]	valid_0's multi_logloss: 0.442435	valid_1's multi_logloss: 0.458794                                       
[76]	valid_0's multi_logloss: 0.441892	valid_1's multi_logloss: 0.458526                                       
[77]	valid_0's multi_logloss: 0.441377	valid_1's multi_logloss: 0.458254                                       
[78]	valid_0's multi_logloss: 0.440873	valid_1's multi_logloss: 0.45801                                        
[79]	valid_0's multi_logloss: 0.440349	valid_1's multi_logloss: 0.457787                                       
[80]	valid_0's multi_logloss: 0.439849	valid_1's multi_logloss: 0.457556                                       
[81]	valid_0's multi_logloss: 0.439343	valid_1's multi_logloss: 0.457322                                

[146]	valid_0's multi_logloss: 0.415146	valid_1's multi_logloss: 0.45013                                       
[147]	valid_0's multi_logloss: 0.414844	valid_1's multi_logloss: 0.450085                                      
[148]	valid_0's multi_logloss: 0.414546	valid_1's multi_logloss: 0.450023                                      
[149]	valid_0's multi_logloss: 0.414252	valid_1's multi_logloss: 0.45002                                       
[150]	valid_0's multi_logloss: 0.413963	valid_1's multi_logloss: 0.449982                                      
[151]	valid_0's multi_logloss: 0.413661	valid_1's multi_logloss: 0.449953                                      
[152]	valid_0's multi_logloss: 0.413363	valid_1's multi_logloss: 0.449902                                      
[153]	valid_0's multi_logloss: 0.413057	valid_1's multi_logloss: 0.449842                                      
[154]	valid_0's multi_logloss: 0.412749	valid_1's multi_logloss: 0.449778                               

...

[146]	valid_0's multi_logloss: 0.415303	valid_1's multi_logloss: 0.450575                                      
[147]	valid_0's multi_logloss: 0.414997	valid_1's multi_logloss: 0.450517                                      
[148]	valid_0's multi_logloss: 0.414697	valid_1's multi_logloss: 0.450459                                      
[149]	valid_0's multi_logloss: 0.414379	valid_1's multi_logloss: 0.450397                                      
[150]	valid_0's multi_logloss: 0.414083	valid_1's multi_logloss: 0.450348                                      
[151]	valid_0's multi_logloss: 0.413791	valid_1's multi_logloss: 0.450292                                      
[152]	valid_0's multi_logloss: 0.413492	valid_1's multi_logloss: 0.450264                                      
[153]	valid_0's multi_logloss: 0.413198	valid_1's multi_logloss: 0.450235                                      
[154]	valid_0's multi_logloss: 0.412894	valid_1's multi_logloss: 0.450198                               

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/home/jindalmadhur26/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "



[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8             
[1]	valid_0's multi_logloss: 0.810459	valid_1's multi_logloss: 0.810719                                        
[2]	valid_0's multi_logloss: 0.774454	valid_1's multi_logloss: 0.774927                                        
[3]	valid_0's multi_logloss: 0.744857	valid_1's multi_logloss: 0.745482                                        
[4]	valid_0's multi_logloss: 0.719447	valid_1's multi_logloss: 0.720233                                        
[5]	valid_0's multi_logloss: 0.697496	valid_1's multi_logloss: 0.698486                                        
[6]	valid_0's multi_logloss: 0.677662	valid_1's multi_logloss: 0.678731                                        
[7]	valid_0's multi_logloss: 0.660357	valid_1's multi_logloss: 0.661558                                        
[8]	valid_0's multi_logloss: 0.644916	valid_1's multi_logloss: 0.646282                                 

[73]	valid_0's multi_logloss: 0.443629	valid_1's multi_logloss: 0.45926                                        
[74]	valid_0's multi_logloss: 0.443084	valid_1's multi_logloss: 0.458994                                       
[75]	valid_0's multi_logloss: 0.442556	valid_1's multi_logloss: 0.458744                                       
[76]	valid_0's multi_logloss: 0.442012	valid_1's multi_logloss: 0.45843                                        
[77]	valid_0's multi_logloss: 0.441486	valid_1's multi_logloss: 0.458162                                       
[78]	valid_0's multi_logloss: 0.440967	valid_1's multi_logloss: 0.457908                                       
[79]	valid_0's multi_logloss: 0.440479	valid_1's multi_logloss: 0.457663                                       
[80]	valid_0's multi_logloss: 0.439966	valid_1's multi_logloss: 0.457391                                       
[81]	valid_0's multi_logloss: 0.439485	valid_1's multi_logloss: 0.457183                                

[146]	valid_0's multi_logloss: 0.415307	valid_1's multi_logloss: 0.450274                                      
[147]	valid_0's multi_logloss: 0.415017	valid_1's multi_logloss: 0.450223                                      
[148]	valid_0's multi_logloss: 0.414723	valid_1's multi_logloss: 0.450171                                      
[149]	valid_0's multi_logloss: 0.414412	valid_1's multi_logloss: 0.450133                                      
[150]	valid_0's multi_logloss: 0.414131	valid_1's multi_logloss: 0.450094                                      
[151]	valid_0's multi_logloss: 0.413824	valid_1's multi_logloss: 0.450078                                      
[152]	valid_0's multi_logloss: 0.413526	valid_1's multi_logloss: 0.450043                                      
[153]	valid_0's multi_logloss: 0.413241	valid_1's multi_logloss: 0.449999                                      
[154]	valid_0's multi_logloss: 0.412936	valid_1's multi_logloss: 0.449919                               